Create Azure SQL Virtual Machine
============================================

Steps of this procedure include:
1. Set variables and set up Notebook 
1. Connect to Azure account and subscription 
1. Configure Network Settings 
1. Provision virtual machine resource in Azure 
1. Provision SQL VM resource in Azure

## Set variables
These variables are set based on your inputs in the deployment wizard. You can make changes to these variables but be aware of possible validation errors caused by your changes.






In [4]:
import os
azure_sqlvm_nb_var_subscription = '5d7346bb-8459-4394-9146-90271743a55f'
azure_sqlvm_nb_var_resource_group_name = 'cloud-shell-storage-centralindia'
azure_sqlvm_location = 'centralindia'
azure_sqlvm_vmname = 'sqlServerVM'
azure_sqlvm_username = 'sqladmin'
azure_sqlvm_image = 'sql2022-ws2022'
azure_sqlvm_image_sku = 'enterprise-gen2'
azure_sqlvm_image_version = '16.0.221025'
azure_sqlvm_vmsize = 'Basic_A1'
azure_sqlvm_newVirtualNetwork = True
azure_sqlvm_virtnet = 'sqlVM'
azure_sqlvm_newSubnet = True
azure_sqlvm_subnet = 'sqlVM'
azure_sqlvm_newPublicIp = True
azure_sqlvm_publicip = 'sqlVM'
azure_sqlvm_allow_rdp = True
azure_sqlvm_sqlConnectivityType = 'public'
azure_sqlvm_port = '1433'
azure_sqlvm_enableSqlAuthentication = True
azure_sqlvm_sqlAuthenticationUsername = 'sqladmin'


## Notebook setup

In [5]:
import sys, json, time, string, random, subprocess

if "AZDATA_NB_VAR_AZURE_SQLVM_PASSWORD" in os.environ:
    azure_sqlvm_password = os.environ["AZDATA_NB_VAR_AZURE_SQLVM_PASSWORD"]

if "AZDATA_NB_VAR_AZURE_SQLVM_SQL_PASSWORD" in os.environ:
    azure_sqlvm_sqlAuthenticationPassword = os.environ["AZDATA_NB_VAR_AZURE_SQLVM_SQL_PASSWORD"]

def run_command(command, json_decode = True, printOutput = True):
    print(command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    if process.returncode != 0: 
        print("Process failed %d \n%s" % (process.returncode, error.decode("utf-8")))
        raise Exception()
    if output:
        output = output.decode("utf-8")
        if printOutput:
            print(output)
        try:
            return json.loads(output)
        except:
            return output

def get_random_string(length):
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    print("Random string of length", length, "is:", result_str)
    return result_str

## Connecting to your Azure account


In [6]:
subscriptions = run_command('az account list', printOutput = False)
if azure_sqlvm_nb_var_subscription not in subscriptions:
    run_command('az login')

az account list


az login



     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A

## Setting your Azure subscription


In [7]:
run_command(
    'az account set '
    '--subscription {0}'
    .format(
        azure_sqlvm_nb_var_subscription));

az account set --subscription 5d7346bb-8459-4394-9146-90271743a55f



     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A

## Configure network settings
All networking configurations are handled in this step, including virtual network, subnet, public IP address, network security group, connectivity settings, and network interface.

1. If you selected the option to create a new virtual network, subnet, or a public IP address, they will be created here. These resources are used to provide network connectivity to the virtual machine and connect it to the internet.

In [8]:
subnet_name = azure_sqlvm_subnet
vnet_name = azure_sqlvm_virtnet
pip_name = azure_sqlvm_publicip


if azure_sqlvm_newVirtualNetwork:
   run_command(
      'az network vnet create '
      '--resource-group {0} '
      '--name {1} '
      '--location {2} '
      '--address-prefixes 192.168.0.0/16 '
      .format(
         azure_sqlvm_nb_var_resource_group_name, 
         vnet_name, 
         azure_sqlvm_location));

if azure_sqlvm_newSubnet:
   run_command(
      'az network vnet subnet create '
      '--name {0} '
      '--resource-group {1} '
      '--vnet-name {2} '
      '--address-prefixes 192.168.1.0/24'
      .format(
         subnet_name,
         azure_sqlvm_nb_var_resource_group_name,
         vnet_name
         ));

if azure_sqlvm_newPublicIp:
   run_command(
      'az network public-ip create '
      '--resource-group {0} '
      '--location {1} '
      '--allocation-method Static '
      '--idle-timeout 4 '
      '--name {2}'
      .format(
         azure_sqlvm_nb_var_resource_group_name, 
         azure_sqlvm_location, 
         pip_name));

az network vnet create --resource-group cloud-shell-storage-centralindia --name sqlVM --location centralindia --address-prefixes 192.168.0.0/16 



     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A


     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A


     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A

2. Create a network security group and configure rules to allow remote desktop (RDP) and SQL Server connections.

In [9]:
nsg_name = azure_sqlvm_nb_var_resource_group_name + 'nsg'
nsg_name_id = run_command(
   'az network nsg create '
   '--resource-group {0} '
   '--location {1} '
   '--name {2} '
   .format(
      azure_sqlvm_nb_var_resource_group_name,
      azure_sqlvm_location, 
      nsg_name));

if azure_sqlvm_allow_rdp:
   run_command(
      'az network nsg rule create '
      '--name RDPRule '
      '--nsg-name {0} '
      '--priority 1000 '
      '--resource-group {1} '
      '--protocol Tcp '
      '--direction Inbound '
      '--source-address-prefixes * '
      '--source-port-range * '
      '--destination-address-prefixes * '
      '--destination-port-range 3389  '
      '--access Allow'
      .format(
         nsg_name,
         azure_sqlvm_nb_var_resource_group_name));

az network nsg create --resource-group cloud-shell-storage-centralindia --location centralindia --name cloud-shell-storage-centralindiansg 



     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A


     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A

3. Create the network interface.

In [10]:
interface_name = azure_sqlvm_nb_var_resource_group_name + "int"

command = (
   'az network nic create '
   '--name {0} '
   '--resource-group {1} '
   '--subnet {2} '
   '--public-ip-address {3} '
   '--network-security-group {4} '
   '--location {5} '
)

if azure_sqlvm_newSubnet:
   command += '--vnet-name {6} '

run_command(
   command
   .format(
      interface_name,
      azure_sqlvm_nb_var_resource_group_name,
      subnet_name,
      pip_name,
      nsg_name,
      azure_sqlvm_location,
      vnet_name));

az network nic create --name cloud-shell-storage-centralindiaint --resource-group cloud-shell-storage-centralindia --subnet sqlVM --public-ip-address sqlVM --network-security-group cloud-shell-storage-centralindiansg --location centralindia --vnet-name sqlVM 



     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A

## Create Azure Virtual Machine and Azure SQL VM resources
First the Azure VM will be created based on all the settings previously specified. Next the SQL VM will be created to include a default set of SQL connectivity settings. The SQL VM resource is where you can manage any SQL Server manageability features offered by Azure. [Learn more](https://docs.microsoft.com/azure/azure-sql/virtual-machines/windows/sql-server-on-azure-vm-iaas-what-is-overview) about what you can do with your Azure SQL VM after it has been created.

In [11]:
# Create the VM
azure_sqlvm_image = 'sql2019-ws2019'

run_command(
   'az vm create '
   '--name {0} '
   '--size {1} '
   '--computer-name {0} '
   '--admin-username {2} '
   '--admin-password {3} '
   '--image {4}:{5}:{6}:{7} '
   '--nics {8} '
   '--resource-group {9} '
   '--location {10} '
   .format(
       azure_sqlvm_vmname,
       azure_sqlvm_vmsize,
       azure_sqlvm_username,
       azure_sqlvm_password,
       'MicrosoftSQLServer',
       azure_sqlvm_image,
       azure_sqlvm_image_sku,
       'latest',
       interface_name,
       azure_sqlvm_nb_var_resource_group_name,
       azure_sqlvm_location
   )
);

az vm create --name sqlServerVM --size Basic_A1 --computer-name sqlServerVM --admin-username sqladmin --admin-password Sqlvm123456789 --image MicrosoftSQLServer:sql2019-ws2019:enterprise-gen2:latest --nics cloud-shell-storage-centralindiaint --resource-group cloud-shell-storage-centralindia --location centralindia 



     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A

In [12]:
command = (
    'az sql vm create '
    '--name {0} '
    '--license-type PAYG '
    '--resource-group {1} '
    '--connectivity-type {2} '
    '--sql-mgmt-type Full '
    '--location {3} '
)

if azure_sqlvm_enableSqlAuthentication:
    command += '--sql-auth-update-username {4} '
    command += '--sql-auth-update-pwd {5} '

if azure_sqlvm_sqlConnectivityType != 'local':
    command += '--port {6} '

run_command(
    command
    .format(
        azure_sqlvm_vmname,
        azure_sqlvm_nb_var_resource_group_name,
        azure_sqlvm_sqlConnectivityType,
        azure_sqlvm_location,
        azure_sqlvm_sqlAuthenticationUsername,
        azure_sqlvm_sqlAuthenticationPassword,
        azure_sqlvm_port,
    )
);

az sql vm create --name sqlServerVM --license-type PAYG --resource-group cloud-shell-storage-centralindia --connectivity-type public --sql-mgmt-type Full --location centralindia --sql-auth-update-username sqladmin --sql-auth-update-pwd Sqlvm123456789 --port 1433 



     /\
    /  \    _____   _ _  ___ _
   / /\ \  |_  / | | | \'__/ _\
  / ____ \  / /| |_| | | |  __/
 /_/    \_\/___|\__,_|_|  \___|


Welcome to the cool new Azure CLI!

Use `az --version` to display the current version.
Here are the base commands:

    account             : Manage Azure subscription information.
    acr                 : Manage private registries with Azure Container Registries.
    ad                  : Manage Azure Active Directory Graph entities needed for Role Based Access
                         Control.
    advisor             : Manage Azure Advisor.
    afd                 : Manage Azure Front Door Standard/Premium. For classical Azure Front Door,
                         please refer https://docs.microsoft.com/en-us/cli/azure/network/front-
                         door?view=azure-cli-latest.
    aks                 : Manage Azure Kubernetes Services.
    ams                 : Manage Azure Media Services resources.
    apim                : Manage Azure A